### 통계의 목적에 따른 분류

#### 1. 기술통계( Descriptive statistics )
- 우리에게 주어진 데이터를 요약, 설명, 분석하는 통계기법을 의미
- pandas ( EDA - 탐색적 데이터 분석 )
- 2가지의 초점
    - 평균(대표값)
    - 분산(데이터의 분포)
    
#### 2. 추론통계( Inferential statistics )
- 수집한 데이터를 기반으로 어떠한 사실을 예측하고 검정하는 통계기법을 의미
- 통계적 가설 검증
    - 표본으로 부터 얻은 어떠한 사실을 근간으로 가설이 맞는지를 통계적으로 검증하는 방법
    - 제일먼저 할일은 가설을 정의하는 것
        - 귀무가설(null hypothesis) => H0
            - 내가 원하는 가설이 아닌, reject가 되는것을 기대하는 가설 
            - 관계가 없다. 영향력이 없다. 관련이 없다.
            - 우리가 채택되기 원하는 가설이 아님
            ![image](https://user-images.githubusercontent.com/28910538/53317646-d3125f00-390f-11e9-8f08-438386aab98e.png)
        - 대립가설 (alternative hypothesis) => H1
            - 관계가 있다. 영향력이 있다. 관계가 있다
            - 통계에 근거해서 채택되기를 원하는 가설


             

![image](https://user-images.githubusercontent.com/28910538/53316320-cb9c8700-390a-11e9-9827-5ae3f078cac1.png)


#### 모집단에 대해서 예측

In [20]:
# import math
import numpy as np

# 모집단 평균 수명 : 75
# 모집단의 표준편차 : 10
# 대립가설 : 의학기술 발달로 평균 수명이 더 높아졌을 것이다
# 귀무가설 : 의학기술 발달이랑 평균수명은 관계없다, 안높아졌을 것이다
# 표본 : 30명
# 표본의 평균수명 : 79
# 유의수준 : 0.1(신뢰도 90%)
last_mean = 75
last_std = 10
num_of_sample = 30
#z = 1.28
z = 2.33
# Z = (CV - 모집단의 평균)/((모집단의 표준편차)/(표본의 수 제곱근))
def Hypo_returnCV(last_mean, last_std, num_of_sample, z):
    return z*(last_std/np.sqrt(num_of_sample))+last_mean
cv = Hypo_returnCV(mo_life_m,mo_life_a,s_num,z)
if  cv > 79:
    # H0
    print("임계값은 : {} 귀무가설 채택, 평균 수명은 변화가 없다".format(cv))
else:
    # H1
    print("임계값은 : {} 귀무가설 기각, 평균 수명은 높아 졌다".format(cv))

임계값은 : 79.2539785299568 귀무가설 채택, 평균 수명은 변화가 없다


### 통계적 가설 검증
- 1. 표본을 통해서 모집단의 특성을 유추
- 2. machine learning을 할 때 필요한 parameter를 분류해서 사용할 필요가 있음
    - 필요없는건 버리고, 서로 연관성이 있는 parameter만 machine learning의 입력으로 넣어줘야 함

##### 카이제곱 검정(Chi-squared Test)

##### 카이제곱 분포표
![image](https://user-images.githubusercontent.com/28910538/53323474-97809080-3921-11e9-9fd7-60852ab78f9c.png)


![image](https://user-images.githubusercontent.com/28910538/53323523-b8e17c80-3921-11e9-877b-976787ab3475.png)


In [171]:
import numpy as np
import pandas as pd
np_zero = np.zeros((4,4), dtype = np.int32)
chi_np_o = np.array([[23,21,63],[31,48,159],[13,23,119]])

for r in range(chi_np_o.shape[0]):
    for c in range(chi_np_o.shape[1]):
        np_zero[r][c] = chi_np_o[r][c]
print(np_zero)


sum_row = np.array([np_zero[r,:].sum() for r in range(np_zero.shape[0])])
sum_col = np.array([np_zero[:,c].sum() for c in range(np_zero.shape[1])])
print(sum_row)
sum_col[-1] = sum_col.sum()
print(sum_col)
# row 의 합을 넣어주고
np_zero[:,-1] = sum_row[:,]
# column의 합을 넣어줌
np_zero[-1,:] = sum_col[:,]
chi_np_o = np_zero
print(chi_np_o)
print("=============================여기까지 dataframe만들기 위한 작업")

df = pd.DataFrame(chi_np_o,
                  index = ["반병 이상", "반병 이하", "못마심", "계"],
                  columns=["1갑 이상", "1갑 이하","안피움", "계"])
display(df)
chi_list_e = []
for r in range(df.shape[0]-1):
    for c in range(df.shape[1]-1):
        chi_list_e.append(((df.iloc[3][c]/(df.iloc[3][3])) * (df.iloc[r][3]/(df.iloc[3][3])))*df.iloc[3][3])
chi_np_e = np.array(chi_list_e)
print("모든 범주에 대한 기대값 계산 ===================")
print(chi_np_e)
print("관측값 - 기대값 ===============================")
chi_np_ome = chi_np_o[:3,:3] - chi_np_e.reshape((3,3))
print(chi_np_ome)
print("(관측값 - 기대값)의 제곱 ======================")
print(chi_np_ome**2)
print("(관측값 - 기대값)의 제곱 / 기대값 ==============")
chi_np_squared = chi_np_ome**2/chi_np_e.reshape((3,3))
print(chi_np_squared)
print("카이제곱 합은 ==============")
print(chi_np_squared.sum())
print("전체 자유도는 =============")
print((chi_np_o.shape[0]-2) * (chi_np_o.shape[1]-2))
if chi_np_squared.sum() > 9.49:
    print("카이제곱 합은 {}, 카이제곱 분포표의 값은 {}".format(chi_np_squared.sum(), 9.49))
    print("따라서 귀무가설은 기각하고 대립가설을 채택, 흡연과 주량은 연관이 있다")
else :
    print("카이제곱 합은 {}, 카이제곱 분포표의 값은 {}".format(chi_np_squared.sum(), 9.49))
    print("따라서 귀무가설을 채택하고 대립가설은 기각, 흡연과 주량은 연관이 없다")

[[ 23  21  63   0]
 [ 31  48 159   0]
 [ 13  23 119   0]
 [  0   0   0   0]]
[107 238 155   0]
[ 67  92 341 500]
[[ 23  21  63 107]
 [ 31  48 159 238]
 [ 13  23 119 155]
 [ 67  92 341 500]]
=============================여기까지 dataframe만들기 위한 작업


,1갑 이상,1갑 이하,안피움,계
반병 이상,23,21,63,107
반병 이하,31,48,159,238
못마심,13,23,119,155
계,67,92,341,500


모든 범주에 대한 기대값 계산 ===================
[ 14.338  19.688  72.974  31.892  43.792 162.316  20.77   28.52  105.71 ]
관측값 - 기대값 ===============================
[[ 8.662  1.312 -9.974]
 [-0.892  4.208 -3.316]
 [-7.77  -5.52  13.29 ]]
(관측값 - 기대값)의 제곱 ======================
[[ 75.030244   1.721344  99.480676]
 [  0.795664  17.707264  10.995856]
 [ 60.3729    30.4704   176.6241  ]]
(관측값 - 기대값)의 제곱 / 기대값 ==============
[[5.23296443 0.08743113 1.36323452]
 [0.0249487  0.40434929 0.06774351]
 [2.90673568 1.0683871  1.67083625]]
카이제곱 합은 ==============
12.826630603041853
전체 자유도는 =============
4
카이제곱 합은 12.826630603041853, 카이제곱 분포표의 값은 9.49
따라서 귀무가설은 기각하고 대립가설을 채택, 흡연과 주량은 연관이 있다


#### matplotlib을 이용한 graph 그리기
- data간의 관계, 방향성등을 눈으로 파악하기 위해서 사용

#### 통계
- 통계는 목적에 따라서 크게 두가지 분류로 구분
    - 1. 기술통계 
        - 가지고 있는 데이터를 분석해서 해당 데이터를 요약, 특정한 사실을 이끌어내는 통계 기법
        - EDA같은 
    - 2. 추론통계
        - 수집한 데이터를 기반으로 모집단의 특성을 유추 독립변수에 따른 종속변수의 변화를 예측하기 위해서 사용하는 통계기법
        - 통계적 가설 검정
            - 1. 표본에 대한 내용을 근거로 모집단의 특성을 유추 (통계적 관점으로의 통계적 가설 검정)
                - 가설 : 귀무가설(연관이없다, 영향이 없다, 효과없다), 대립가설(귀무가설에 반대되는 목표)
            - 2. 또 다른 목적으로 통계적 가설 검정을 이용함(기계학습 관점으로의 통계적 가설 검정)
                - 머신러닝을 위해 여러가지 입력 parameter들이 들어 가는데, 어떤 parameter는 학습하는데 필요하고, 또 어떤 parameter는 필요가 없음, parameter간의 관계성을 고려해서 입력 parameter를 설정해주는 작업이 필요 통계적 가설 검정을 통해 parameter간의 관련성을 검증하고 필요한 parameter만을 뽑아 입력값으로 사용
                - 의미 있는 변수 찾기 위해 여러 검정 방법이 있는데 그 중
                    - 카이제곱 검정(Chi-squared Test)
                        - 범주형으로 되어있고 밀도값이 데이터로 사용될 때 사용되는 검정
                    

### 독립표본 t검정
- 두 집단의 평균을 이용해서 두 집단이 서로 차이가 있는지를 판단하는 검정방법
    - 그룹 1과 그룹2의 평균키를 조사해서 그 차이가 있는지(의미가 있는지) 혹은 차이를 무시할 정도인지를 판별 
    - 1. 통계적 가설검정은 가설검정부터
        - 귀무가설 : 두 그룹간의 평균키의 차이는 의미가 없다
        - 대립가설 : 두 그룹간의 평균키의 차이는 의미가 있다
- scipy에서 status 사용
    - 독립 표본, 독립관계니까 independent의 stats.ttest_ind 사용

In [30]:
import numpy as np
from scipy import stats

# 일반적으로 사용하지 않는 변수는 "_"
group1 = [170+ np.random.normal(2,1) for _ in range(10)]
group2 = [174+ np.random.normal(0,3) for _ in range(10)]

print("Group1의 평균 : {}".format(np.mean(group1)))
print("Group2의 평균 : {}".format(np.mean(group2)))


# t-검정
_, pvalue = stats.ttest_ind(group1, group2)
print(pvalue)
# pvalue 값으로 판단, 0.05보다 작다면 95% 신뢰도에서 귀무가설 기각, 대립가설 채택
if pvalue < 0.05:
    print("대립가설 채택 : 평균키의 차이는 의미가 있다.")
else:
    print("귀무가설 채택 : 평균키의 차이는 의미가 없다.")

Group1의 평균 : 172.0992799810142
Group2의 평균 : 174.70893470067097
0.0002454479376783881
대립가설 채택 : 평균키의 차이는 의미가 있다.


### 대응표본 t검정
- 다이어트약의 복용전과 복용후의 값을 통계적으로 검증하여 약의 효과를 알아보기 위해서 사용
- 귀무가설 : 복용전후의 체중의 차이가 없다
- 대립가설 : 복용전후의 체중의 차이가 있다
- 표본간 독립하지않고 서로 관계를 갖고 있기 때문에 relation, scipy의 status.ttest_rel 사용

In [65]:
import numpy as np
from scipy import stats

before = [60 + np.random.normal(0,5) for _ in range(20)]
after = [w - np.random.normal(2,1) for w in before]


_, pvalue = stats.ttest_rel(before,after)

print(pvalue)
if pvalue < 0.05:
    print("대립가설이 채택 : 다이어트 약이 효과가 있음")
else :
    print("귀무가설이 채택 : 다이어트 약이 효과가 없음")

2.792741462822641e-06
대립가설이 채택 : 다이어트 약이 효과가 있음


### ANOVA 검정
- 비교대상이 3개 이상일 경우 ttest대신 사용
- 대응표본 t-검정의 연장선

In [68]:
from scipy import stats

a = [67,45,98,67,34,22]
b = [56,48,80,37,34,31]
c = [80,61,71,67,44,15]
d = [79,31,61,80,70,47]

_, pvalue = stats.f_oneway(a,b,c,d)
print(pvalue)

0.7688091788784993
